In [1]:
# required Packages and Modules 
# !pip install opencv-python
# !pip install matplotlib
# !pip install tensorflow
# !pip install scikit-learn

#Uncomment the above pip commands to install the packages

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import PIL

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.callbacks import Callback


In [3]:
#Creating a path object 
data_dir = Path("./Dataset/BHSig260-Hindi") #Change the path as needed

# Labeling the images
images, labels = [],[]

for dir in data_dir.iterdir():
    if dir.is_dir():
        for i in dir.glob('*.tif'):
            label = 0 if 'G' in i.stem else 1
            images.append(i)
            labels.append(label)

In [4]:
#Function for preprocessing each image
def load_and_preprocess_image(path, label):
    def _load_image(path, label):
        path = path.numpy().decode('utf-8')
        #Resizing the image to the required image size for mobilenet(224x224)
        image = tf.keras.preprocessing.image.load_img(path, target_size=(224, 224))
        image = tf.keras.preprocessing.image.img_to_array(image)
        #Preprocessing 
        image = preprocess_input(image)
        return image, label

    [image, label] = tf.py_function(_load_image, [path, label], [tf.float32, tf.int32])
    image.set_shape((224, 224, 3))
    label.set_shape([])  # Ensure label is a scalar
    return image, label


In [5]:
# Example: Assuming 'images' is a list of image file paths (as strings) and 'labels' are the corresponding labels
# First, ensure your 'images' list contains string paths, not PosixPath objects
images = [str(img_path) for img_path in images]  # Convert PosixPath to string if necessary

In [6]:
# Split the data into train and temp sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, train_size=0.7, stratify=labels)

# Split the temp set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)


In [7]:
#Conversion to tensorflow dataset, Preprocess each image and shuffle the training dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
#Conversion to tensorflow dataset, Preprocess each image and shuffle the validation dataset
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
#Conversion to tensorflow dataset, Preprocess each image and shuffle the test dataset
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

The Preprocessing stage ended

In [10]:
print(len(train_ds), len(val_ds), len(test_ds))

def check_shapes(image, label):
    print("Image shape:", image.shape, "Label:", label)
    return image, label

train_ds = train_ds.map(check_shapes)
val_ds = val_ds.map(check_shapes)


189 41 41
Image shape: (None, 224, 224, 3) Label: Tensor("args_1:0", shape=(None,), dtype=int32)
Image shape: (None, 224, 224, 3) Label: Tensor("args_1:0", shape=(None,), dtype=int32)


In [11]:


# Load the MobileNetV3Small model, excluding the top fully connected layer
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Set all layers of the base model to be trainable
for layer in base_model.layers:
    layer.trainable = True

# Adding custom layers
x = base_model.output
x = Flatten()(x)  # Flatten the output
predictions = Dense(1, activation='sigmoid')(x)  # Final dense layer for binary classification

# Creating the final model
model = Model(inputs=base_model.input, outputs=predictions)

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


f1_metric = F1Score()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), f1_metric, AUC()])

# Train the model
history = model.fit(train_ds, epochs=20, validation_data=val_ds)


Epoch 1/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 206s 931ms/step - accuracy: 0.6753 - auc: 0.7275 - f1_score: 0.7095 - loss: 0.7427 - precision_1: 0.7024 - recall_1: 0.7180 - val_accuracy: 0.6397 - val_auc: 0.7267 - val_f1_score: 0.7296 - val_loss: 0.7101 - val_precision_1: 0.6256 - val_recall_1: 0.8750
Epoch 2/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 190s 886ms/step - accuracy: 0.8616 - auc: 0.9319 - f1_score: 0.8755 - loss: 0.3307 - precision_1: 0.8709 - recall_1: 0.8803 - val_accuracy: 0.6713 - val_auc: 0.7372 - val_f1_score: 0.7351 - val_loss: 0.6372 - val_precision_1: 0.6655 - val_recall_1: 0.8208
Epoch 3/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 170s 887ms/step - accuracy: 0.9259 - auc: 0.9772 - f1_score: 0.9331 - loss: 0.2028 - precision_1: 0.9263 - recall_1: 0.9401 - val_accuracy: 0.6721 - val_auc: 0.7708 - val_f1_score: 0.7475 - val_loss: 0.6532 - val_precision_1: 0.6532 - val_recall_1: 0.8736
Epoch 4/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 203s 893ms/step - accuracy: 0.9683 - auc: 0.9950 - f1_score: 0.9714 - lo

2024-04-22 11:17:29.157453: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 940 of 1000
2024-04-22 11:17:29.613299: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


189/189 ━━━━━━━━━━━━━━━━━━━━ 177s 877ms/step - accuracy: 0.9999 - auc: 1.0000 - f1_score: 0.9999 - loss: 0.0016 - precision_1: 0.9998 - recall_1: 1.0000 - val_accuracy: 0.7670 - val_auc: 0.8372 - val_f1_score: 0.7855 - val_loss: 0.6135 - val_precision_1: 0.8038 - val_recall_1: 0.7681
Epoch 19/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 182s 928ms/step - accuracy: 0.9996 - auc: 1.0000 - f1_score: 0.9996 - loss: 0.0027 - precision_1: 1.0000 - recall_1: 0.9992 - val_accuracy: 0.7832 - val_auc: 0.8606 - val_f1_score: 0.7892 - val_loss: 0.6169 - val_precision_1: 0.8581 - val_recall_1: 0.7306
Epoch 20/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 180s 930ms/step - accuracy: 1.0000 - auc: 1.0000 - f1_score: 1.0000 - loss: 0.0012 - precision_1: 1.0000 - recall_1: 1.0000 - val_accuracy: 0.7693 - val_auc: 0.8662 - val_f1_score: 0.7662 - val_loss: 0.6824 - val_precision_1: 0.8766 - val_recall_1: 0.6806


In [12]:
model.save("MobileNetV3Small_without_augmentation.keras")

In [17]:


# Custom F1-Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


model = load_model('MobileNetV3Small_without_augmentation.keras', custom_objects={'F1Score': F1Score})

evaluation_results = model.evaluate(test_ds)
print(f"Test Loss: {evaluation_results[0]:.3f}")

# Since the first value is always the loss, you can print out the rest as metrics
for i, metric in enumerate(model.metrics_names[1:], 1):
    print(f"{metric}: {evaluation_results[i]:.3f}")
print(evaluation_results)

41/41 ━━━━━━━━━━━━━━━━━━━━ 12s 215ms/step - accuracy: 0.7395 - auc: 0.8590 - f1_score: 0.7308 - loss: 0.6726 - precision_1: 0.8509 - recall_1: 0.6406
Test Loss: 0.727
compile_metrics: 0.735
[0.7273659110069275, 0.7353395223617554, 0.8563327193260193, 0.6291666626930237, 0.7253802418708801, 0.854687511920929]
